In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
%matplotlib inline
from itertools import combinations, product
from sklearn.model_selection import KFold,cross_val_predict,cross_val_score, cross_validate
from sklearn.linear_model import LinearRegression,RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from statsmodels.formula.api import ols
import statsmodels.api as sm
import scipy.stats as stats
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
df=pd.read_excel('2019 County Health Rankings Data - v2.xls',3,header=1)

df_addtl = pd.read_excel('2019 County Health Rankings Data - v2.xls',5,header=1)

xls = pd.ExcelFile('unemployment.xls')

unemployment = xls.parse('Unemployment Med HH Inc', skiprows=4, index_col=None, na_values=['NA'])


In [3]:
df_addtl = df_addtl[['FIPS',
                     '% Frequent Physical Distress',
                     '% Frequent Mental Distress',
                     '% Food Insecure',
                     '% Limited Access',
                     '% Insufficient Sleep',
                     '% Uninsured',
                     '% Homeowners',
                     '% Severe Housing Cost Burden',
                     'Population'
                    ]]

In [4]:
df_addtl.head()

,FIPS,% Frequent Physical Distress,% Frequent Mental Distress,% Food Insecure,% Limited Access,% Insufficient Sleep,% Uninsured,% Homeowners,% Severe Housing Cost Burden,Population
0,1001,12.824434,12.638357,13.4,11.991125,35.905406,10.961104,73.287736,12.645206,55504
1,1003,12.622002,12.822558,12.3,5.424427,33.305868,13.570311,72.859338,12.795922,212628
2,1005,16.216160,14.632311,23.2,10.739667,38.563167,16.542240,62.506800,13.586765,25270
3,1007,13.162968,12.854036,15.8,2.601627,38.148865,12.297319,75.361481,8.733889,22668
4,1009,13.720057,13.827943,11.0,2.880150,35.945010,15.564604,78.521025,8.194495,58013


In [5]:
unemployment.head()

,FIPS,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2007,Employed_2007,Unemployed_2007,Unemployment_rate_2007,Civilian_labor_force_2008,Employed_2008,Unemployed_2008,Unemployment_rate_2008,Civilian_labor_force_2009,Employed_2009,Unemployed_2009,Unemployment_rate_2009,Civilian_labor_force_2010,Employed_2010,Unemployed_2010,Unemployment_rate_2010,Civilian_labor_force_2011,Employed_2011,Unemployed_2011,Unemployment_rate_2011,Civilian_labor_force_2012,Employed_2012,Unemployed_2012,Unemployment_rate_2012,Civilian_labor_force_2013,Employed_2013,Unemployed_2013,Unemployment_rate_2013,Civilian_labor_force_2014,Employed_2014,Unemployed_2014,Unemployment_rate_2014,Civilian_labor_force_2015,Employed_2015,Unemployed_2015,Unemployment_rate_2015,Civilian_labor_force_2016,Employed_2016,Unemployed_2016,Unemployment_rate_2016,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018,Unemployed_avg(2007-2018)
0,0,US,United States,NaN,NaN,NaN,152191093.0,145156134.0,7034959.0,4.6,153761095.0,144860350.0,8900745.0,5.8,153825455.0,139594698.0,14230757.0,9.3,154270732.0,139408204.0,14862528.0,9.63,154606316.0,140765809.0,13840507.0,8.95,155119195.0,142600402.0,12518793.0,8.07,155485692.0,144018151.0,11467541.0,7.38,155957297.0,146338305.0,9618992.0,6.17,156866585.0,148579026.0,8287559.0,5.28,158708795.0,150981151.0,7727644.0,4.87,160273906.0,153293830.0,6980076.0,4.36,161640488.0,155344043.0,6296445.0,3.9,61937.0,NaN,NaN
1,1000,AL,Alabama,NaN,NaN,NaN,2175612.0,2089127.0,86485.0,4.0,2176489.0,2053477.0,123012.0,5.7,2162999.0,1924747.0,238252.0,11.0,2196042.0,1964559.0,231483.0,10.50,2202670.0,1990413.0,212257.0,9.60,2176337.0,2003290.0,173047.0,8.00,2174000.0,2017043.0,156957.0,7.20,2160842.0,2014290.0,146552.0,6.80,2158293.0,2026898.0,131395.0,6.10,2177209.0,2049971.0,127238.0,5.80,2178243.0,2081676.0,96567.0,4.40,2198837.0,2112347.0,86490.0,3.9,49881.0,100.0000,6.916667
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,24383.0,23577.0,806.0,3.3,24687.0,23420.0,1267.0,5.1,24703.0,22301.0,2402.0,9.7,25713.0,23431.0,2282.0,8.90,25836.0,23677.0,2159.0,8.40,25740.0,23961.0,1779.0,6.90,25810.0,24205.0,1605.0,6.20,25592.0,24097.0,1495.0,5.80,25613.0,24283.0,1330.0,5.20,25966.0,24645.0,1321.0,5.10,25972.0,24958.0,1014.0,3.90,25957.0,25015.0,942.0,3.6,59338.0,118.9591,6.008333
3,1003,AL,"Baldwin County, AL",3.0,2.0,1.0,82659.0,80099.0,2560.0,3.1,83223.0,79372.0,3851.0,4.6,82451.0,74403.0,8048.0,9.8,83459.0,75120.0,8339.0,10.00,85045.0,77418.0,7627.0,9.00,84414.0,78065.0,6349.0,7.50,85280.0,79626.0,5654.0,6.60,86384.0,81083.0,5301.0,6.10,87741.0,82882.0,4859.0,5.50,90670.0,85839.0,4831.0,5.30,92090.0,88342.0,3748.0,4.10,93849.0,90456.0,3393.0,3.6,57588.0,115.4508,6.266667
4,1005,AL,"Barbour County, AL",6.0,6.0,0.0,10334.0,9684.0,650.0,6.3,10161.0,9267.0,894.0,8.8,10003.0,8572.0,1431.0,14.3,10221.0,8959.0,1262.0,12.30,9849.0,8712.0,1137.0,11.50,9362.0,8283.0,1079.0,11.50,9099.0,8168.0,931.0,10.20,8845.0,7913.0,932.0,10.50,8613.0,7848.0,765.0,8.90,8417.0,7717.0,700.0,8.30,8317.0,7831.0,486.0,5.80,8373.0,7940.0,433.0,5.2,34382.0,68.9280,9.466667


In [6]:
unemployment = unemployment[['FIPS','Unemployment_rate_2018','Median_Household_Income_2018',
                            'Unemployed_avg(2007-2018)']]
unemployment.head()

,FIPS,Unemployment_rate_2018,Median_Household_Income_2018,Unemployed_avg(2007-2018)
0,0,3.9,61937.0,NaN
1,1000,3.9,49881.0,6.916667
2,1001,3.6,59338.0,6.008333
3,1003,3.6,57588.0,6.266667
4,1005,5.2,34382.0,9.466667


In [7]:
df=df[['FIPS','State', 'County',
#            'Violent Crime Rate',
#            'Years of Potential Life Lost Rate',
           '% Fair/Poor',
           'Physically Unhealthy Days',
           'Mentally Unhealthy Days',
#            '% LBW',
           '% Smokers',
           '% Obese',
           'Food Environment Index',
           '% Physically Inactive',
           '% With Access',
           '% Excessive Drinking',
#            '% Alcohol-Impaired', 
#            'Chlamydia Rate',
#            'Teen Birth Rate',
#            '% Uninsured',
           'PCP Rate',
           'Dentist Rate',
           'MHP Rate',
#            'Preventable Hosp. Rate',
#            '% Screened',
#            '% Vaccinated',
           'Graduation Rate',
           '% Some College',
#            '% Unemployed',
#            '% Children in Poverty',
           'Income Ratio',
#            '% Single-Parent Households',
           'Association Rate',
#            'Injury Death Rate',
           'Average Daily PM2.5',
           'Presence of violation',
           '% Severe Housing Problems',
           '% Drive Alone',
           '% Long Commute - Drives Alone'
          ]]
df.head(3)

,FIPS,State,County,% Fair/Poor,Physically Unhealthy Days,Mentally Unhealthy Days,% Smokers,% Obese,Food Environment Index,% Physically Inactive,% With Access,% Excessive Drinking,PCP Rate,Dentist Rate,MHP Rate,Graduation Rate,% Some College,Income Ratio,Association Rate,Average Daily PM2.5,Presence of violation,% Severe Housing Problems,% Drive Alone,% Long Commute - Drives Alone
0,1001,Alabama,Autauga,18.411124,4.200578,4.306739,19.124658,37.5,7.2,31.1,68.677503,16.878015,41.50426,32.43010,16.21505,90.000000,60.854665,4.636664,12.631731,11.7,No,14.954646,85.965057,38.3
1,1003,Alabama,Baldwin,18.060458,4.098748,4.249649,16.795485,31.0,8.0,23.8,71.971031,16.714969,72.87966,49.85232,91.23916,86.361577,66.133926,4.518560,10.692213,10.3,Yes,13.831725,84.719423,40.5
2,1005,Alabama,Barbour,25.773416,5.067438,4.634994,21.540878,44.3,5.6,28.2,53.625669,12.698715,38.51338,35.61535,7.91452,81.410256,36.774008,5.755063,8.472944,11.5,No,15.455531,83.404353,33.8


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 24 columns):
FIPS                             3142 non-null int64
State                            3142 non-null object
County                           3142 non-null object
% Fair/Poor                      3142 non-null float64
Physically Unhealthy Days        3142 non-null float64
Mentally Unhealthy Days          3142 non-null float64
% Smokers                        3142 non-null float64
% Obese                          3142 non-null float64
Food Environment Index           3123 non-null float64
% Physically Inactive            3142 non-null float64
% With Access                    3136 non-null float64
% Excessive Drinking             3142 non-null float64
PCP Rate                         3007 non-null float64
Dentist Rate                     3057 non-null float64
MHP Rate                         2890 non-null float64
Graduation Rate                  3043 non-null float64
% Some College   

In [9]:
df_2 = (pd.merge(df,df_addtl, how='left', on=['FIPS']))

df_new = pd.merge(df_2, unemployment, how='left', on=['FIPS'])

In [10]:
df_new.head()

,FIPS,State,County,% Fair/Poor,Physically Unhealthy Days,Mentally Unhealthy Days,% Smokers,% Obese,Food Environment Index,% Physically Inactive,% With Access,% Excessive Drinking,PCP Rate,Dentist Rate,MHP Rate,Graduation Rate,% Some College,Income Ratio,Association Rate,Average Daily PM2.5,Presence of violation,% Severe Housing Problems,% Drive Alone,% Long Commute - Drives Alone,% Frequent Physical Distress,% Frequent Mental Distress,% Food Insecure,% Limited Access,% Insufficient Sleep,% Uninsured,% Homeowners,% Severe Housing Cost Burden,Population,Unemployment_rate_2018,Median_Household_Income_2018,Unemployed_avg(2007-2018)
0,1001,Alabama,Autauga,18.411124,4.200578,4.306739,19.124658,37.5,7.2,31.1,68.677503,16.878015,41.50426,32.43010,16.21505,90.000000,60.854665,4.636664,12.631731,11.7,No,14.954646,85.965057,38.3,12.824434,12.638357,13.4,11.991125,35.905406,10.961104,73.287736,12.645206,55504,3.6,59338.0,6.008333
1,1003,Alabama,Baldwin,18.060458,4.098748,4.249649,16.795485,31.0,8.0,23.8,71.971031,16.714969,72.87966,49.85232,91.23916,86.361577,66.133926,4.518560,10.692213,10.3,Yes,13.831725,84.719423,40.5,12.622002,12.822558,12.3,5.424427,33.305868,13.570311,72.859338,12.795922,212628,3.6,57588.0,6.266667
2,1005,Alabama,Barbour,25.773416,5.067438,4.634994,21.540878,44.3,5.6,28.2,53.625669,12.698715,38.51338,35.61535,7.91452,81.410256,36.774008,5.755063,8.472944,11.5,No,15.455531,83.404353,33.8,16.216160,14.632311,23.2,10.739667,38.563167,16.542240,62.506800,13.586765,25270,5.2,34382.0,9.466667
3,1007,Alabama,Bibb,19.996912,4.363377,4.315710,19.916404,37.8,7.6,34.9,16.251364,15.925151,57.41289,26.46903,8.82301,83.763838,48.406310,4.254594,10.157665,11.2,No,10.960854,86.365902,48.6,13.162968,12.854036,15.8,2.601627,38.148865,12.297319,75.361481,8.733889,22668,4.0,46064.0,7.508333
4,1009,Alabama,Blount,21.095323,4.512753,4.701599,19.652158,34.4,8.5,28.5,22.949304,15.378722,22.52877,20.68502,10.34251,93.468795,54.413998,4.126883,9.011507,11.7,No,10.401153,86.792275,59.7,13.720057,13.827943,11.0,2.880150,35.945010,15.564604,78.521025,8.194495,58013,3.5,50412.0,6.166667


In [11]:
df_new.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 0 to 3141
Data columns (total 36 columns):
FIPS                             3142 non-null int64
State                            3142 non-null object
County                           3142 non-null object
% Fair/Poor                      3142 non-null float64
Physically Unhealthy Days        3142 non-null float64
Mentally Unhealthy Days          3142 non-null float64
% Smokers                        3142 non-null float64
% Obese                          3142 non-null float64
Food Environment Index           3123 non-null float64
% Physically Inactive            3142 non-null float64
% With Access                    3136 non-null float64
% Excessive Drinking             3142 non-null float64
PCP Rate                         3007 non-null float64
Dentist Rate                     3057 non-null float64
MHP Rate                         2890 non-null float64
Graduation Rate                  3043 non-null float64
% Some College   

In [12]:
df_new.isnull().sum()

FIPS                               0
State                              0
County                             0
% Fair/Poor                        0
Physically Unhealthy Days          0
Mentally Unhealthy Days            0
% Smokers                          0
% Obese                            0
Food Environment Index            19
% Physically Inactive              0
% With Access                      6
% Excessive Drinking               0
PCP Rate                         135
Dentist Rate                      85
MHP Rate                         252
Graduation Rate                   99
% Some College                     0
Income Ratio                       0
Association Rate                   0
Average Daily PM2.5               34
Presence of violation             43
% Severe Housing Problems          0
% Drive Alone                      0
% Long Commute - Drives Alone      0
% Frequent Physical Distress       0
% Frequent Mental Distress         0
% Food Insecure                    0
%

In [19]:
column_isna_list = ['Food Environment Index',
                    '% With Access',
               'PCP Rate',
                   'Dentist Rate',
                   'MHP Rate',
                    'Graduation Rate',
                    'Average Daily PM2.5',
                    '% Limited Access'] 

for column in column_isna_list:
    df_new[column] = df_new.groupby("State").transform(lambda x: x.fillna(x.mean()))
    
df_new['Presence of violation']=df_new['Presence of violation'].apply(lambda x: 1 if x=='Yes' else 0)

# df_new["Graduation Rate"] = df_new.groupby("State").transform(lambda x: x.fillna(x.mean()))
df_new.isna().sum()

FIPS                             0
State                            0
County                           0
% Fair/Poor                      0
Physically Unhealthy Days        0
Mentally Unhealthy Days          0
% Smokers                        0
% Obese                          0
Food Environment Index           0
% Physically Inactive            0
% With Access                    0
% Excessive Drinking             0
PCP Rate                         0
Dentist Rate                     0
MHP Rate                         0
Graduation Rate                  0
% Some College                   0
Income Ratio                     0
Association Rate                 0
Average Daily PM2.5              0
Presence of violation            0
% Severe Housing Problems        0
% Drive Alone                    0
% Long Commute - Drives Alone    0
% Frequent Physical Distress     0
% Frequent Mental Distress       0
% Food Insecure                  0
% Limited Access                 0
% Insufficient Sleep

In [20]:
df1 = df_new[df_new.isna().any(axis=1)]

df1

,FIPS,State,County,% Fair/Poor,Physically Unhealthy Days,Mentally Unhealthy Days,% Smokers,% Obese,Food Environment Index,% Physically Inactive,% With Access,% Excessive Drinking,PCP Rate,Dentist Rate,MHP Rate,Graduation Rate,% Some College,Income Ratio,Association Rate,Average Daily PM2.5,Presence of violation,% Severe Housing Problems,% Drive Alone,% Long Commute - Drives Alone,% Frequent Physical Distress,% Frequent Mental Distress,% Food Insecure,% Limited Access,% Insufficient Sleep,% Uninsured,% Homeowners,% Severe Housing Cost Burden,Population,Unemployment_rate_2018,Median_Household_Income_2018,Unemployed_avg(2007-2018)
548,15005,Hawaii,Kalawao,14.017149,3.214459,3.171273,12.671810,22.8,15005.0,20.5,15005.0,16.653588,15005.0,15005.0,15005.0,15005.0,81.250000,5.477273,0.0,15005.0,0,18.181818,32.812500,0.0,9.636388,9.996988,18.8,15005.0,38.328812,NaN,3.773585,6.666667,88,NaN,NaN,NaN
2673,48301,Texas,Loving,18.370342,3.618747,3.680144,19.109575,28.8,48301.0,22.2,48301.0,19.219299,48301.0,48301.0,48301.0,48301.0,58.333333,2.628295,0.0,48301.0,0,17.777778,71.794872,21.4,11.242246,11.759326,18.7,48301.0,32.594713,21.052632,51.612903,NaN,134,3.9,78637.0,5.875
